# Ranked Retrieval with tf-idf

In [1]:
from functools import total_ordering, reduce
from collections import defaultdict
from math import log
import csv
import re

### Postings

In [2]:
@total_ordering
class Posting:
    
    def __init__(self, docID, tf):
        self._docID = docID
        self._tf = tf  # Term frequency
        
    def get_from_corpus(self, corpus):
        return corpus[self._docID]
    
    def __eq__(self, other):
        return self._docID == other._docID
    
    def __gt__(self, other):
        return self._docID > other._docID
    
    def __repr__(self):
        return str(self._docID) + " [" + str(self._tf) + "]"
    
    def __hash__(self):
        return hash(self._docID)

### Posting Lists

In [3]:
class PostingList:
    
    def __init__(self):
        self._postings = []
    
    @classmethod
    def from_docID(cls, docID, tf):
        plist = cls()
        plist._postings = [(Posting(docID, tf))]
        return plist
    
    @classmethod
    def from_posting_list(cls, postingList):
        plist = cls()
        plist._postings = postingList
        return plist
    
    def merge(self, other):
        i = 0
        last = self._postings[-1]
        while (i < len(other._postings) and last == other._postings[i]):
            last._tf += other._postings[i]._tf
            i += 1
        self._postings += other._postings[i:]
        
    def intersection(self, other):
        intersection = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other._postings)):
            if (self._postings[i] == other._postings[j]):
                intersection.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                i += 1
            else:
                j += 1
        return PostingList.from_posting_list(intersection)
    
    def union(self, other):
        union = []
        i = 0
        j = 0
        while (i < len(self._postings) and j < len(other.postings)):
            if (self._postings[i] == other._postings[j]):
                union.append(self._postings[i])
                i += 1
                j += 1
            elif (self._postings[i] < other._postings[j]):
                union.append(self._postings[i])
                i += 1
            else:
                union.append(other._postings[j])
                j += 1
        for k in range(i, len(self._postings)):
            union.append(self._postings[k])
        for k in range(j, len(other._postings)):
            union.append(other._postings[k])
        return PostingList.from_posting_list(union)
    
    def get_from_corpus(self, corpus):
        return list(map(lambda x: x.get_from_corpus(corpus), self._postings))
    
    def __len__(self):
        return len(self._postings)

    def __iter__(self):
        return iter(self._postings)
    
    def __repr__(self):
        return ", ".join(map(str, self._postings))

### Terms

In [4]:
class ImpossibleMergeError(Exception):
    pass

@total_ordering
class Term:
    
    def __init__(self, term, docID, tf):
        self.term = term
        self.posting_list = PostingList.from_docID(docID, tf)
        
    def merge(self, other):
        if (self.term == other.term):
            self.posting_list.merge(other.posting_list)
        else:
            raise ImpossibleMergeError
    
    def __eq__(self, other):
        return self.term == other.term
    
    def __gt__(self, other):
        return self.term > other.term
    
    def __repr__(self):
        return self.term + ": " + repr(self.posting_list)

### Inverted Index

In [18]:
def normalize(text):
    no_punctuation = re.sub(r'[^\w^\s^-]','',text)
    downcase = no_punctuation.lower()
    return downcase

def tokenize(movie):
    text = normalize(movie.description)
    return list(text.split())

class InvertedIndex:
    
    def __init__(self):
        self._dictionary = []
        self._N = 0  # Number of documents in the collection
        
    @classmethod
    def from_corpus(cls, corpus):
        intermediate_dict = {}
        for docID, document in enumerate(corpus):
            tokens = tokenize(document)
            for token in tokens:
                term = Term(token, docID, 1)
                try:
                    intermediate_dict[token].merge(term)
                except KeyError:
                    intermediate_dict[token] = term
            if (docID % 1000 == 0):
                print("ID: " + str(docID))
        idx = cls()
        idx._N = len(corpus)
        idx._dictionary = sorted(intermediate_dict.values())
        return idx
    
    def __getitem__(self, key):
        for term in self._dictionary:
            if term.term == key:
                return term.posting_list
        raise KeyError
        
    def __repr__(self):
        return "A dictionary with " + str(len(self._dictionary)) + " terms"

### Reading the Corpus

In [6]:
class MovieDescription:
    
    def __init__(self, title, description):
        self.title = title
        self.description = description
        
    def __repr__(self):
        return self.title
    
def read_movie_descriptions():
    filename = 'plot_summaries.txt'
    movie_names_file = 'movie.metadata.tsv'
    with open(movie_names_file, 'r',encoding="utf8") as csv_file:
        movie_names = csv.reader(csv_file, delimiter='\t')
        names_table = {}
        for name in movie_names:
            names_table[name[0]] = name[2]
    with open(filename, 'r',encoding="utf8") as csv_file:
        descriptions = csv.reader(csv_file, delimiter='\t')
        corpus = []
        for desc in descriptions:
            try:
                movie = MovieDescription(names_table[desc[0]], desc[1])
                corpus.append(movie)
            except KeyError:
                pass
        return corpus

In [7]:
corpus = read_movie_descriptions()

In [8]:
len(corpus)

42204

In [9]:
index = InvertedIndex.from_corpus(corpus)

ID: 0
ID: 1000
ID: 2000
ID: 3000
ID: 4000
ID: 5000
ID: 6000
ID: 7000
ID: 8000
ID: 9000
ID: 10000
ID: 11000
ID: 12000
ID: 13000
ID: 14000
ID: 15000
ID: 16000
ID: 17000
ID: 18000
ID: 19000
ID: 20000
ID: 21000
ID: 22000
ID: 23000
ID: 24000
ID: 25000
ID: 26000
ID: 27000
ID: 28000
ID: 29000
ID: 30000
ID: 31000
ID: 32000
ID: 33000
ID: 34000
ID: 35000
ID: 36000
ID: 37000
ID: 38000
ID: 39000
ID: 40000
ID: 41000
ID: 42000


### Putting it all together

In [7]:
class IRsystem:
    
    def __init__(self, corpus, index):
        self._corpus = corpus
        self._index = index
        
    @classmethod
    def from_corpus(cls, corpus):
        index = InvertedIndex.from_corpus(corpus)
        return cls(corpus, index)
    
    def answer_query(self, words): # ['cat', 'batman']
        norm_words = map(normalize, words)
        postings = map(lambda w: self._index[w], norm_words)
        plist = reduce(lambda x, y: x.intersection(y), postings)
        return plist.get_from_corpus(self._corpus)
    
    # New methods, this time with spelling correction!
    def answer_query_sc(self, words):
        norm_words = map(normalize, words)
        postings = []
        for w in norm_words:
            try:
                res = self._index[w]
            except KeyError:
                dictionary = [t.term for t in self._index._dictionary]
                sub = find_nearest(w, dictionary, keep_first=True)
                print("{} not found. Did you mean {}?".format(w, sub))
                res = self._index[sub]
            postings.append(res)
        plist = reduce(lambda x, y: x.intersection(y), postings)
        return plist.get_from_corpus(self._corpus)
    
    def answer_query_weighted(self, words):
        norm_words = [normalize(w) for w in words]
        scores = defaultdict(lambda: 0)
        for w in norm_words:
            try:
                res = self._index[w]
            except KeyError:
                dictionary = [t.term for i in self._index._dictionary]
                sub = find_nearest(w, dictionary, keep_first=True)
                print("{} not found. Did you mean {}?".format(w, sub))
                w = sub
                res = self._index[w]
            idf_w = log(self._index._N/len(self._index[w]._postings))
            for doc in res:
                scores[doc] += idf_w * doc._tf
        plist = sorted(scores.items(), key=lambda x: -x[1])
        return [(x[0].get_from_corpus(self._corpus), x[1]) for x in plist]
            
    
def query(ir, text):
    words = text.split()
    answer = ir.answer_query(words)
    for movie in answer:
        print(movie)
        
def query_sc(ir, text):
    words = text.split()
    answer = ir.answer_query_sc(words)
    for movie in answer:
        print(movie)
        
def query_weighted(ir, text):
    words = text.split()
    answer = ir.answer_query_weighted(words)
    for movie in answer:
        print(movie)

### Edit Distance

In [8]:
def edit_distance(u, v):
    nrows = len(u) + 1
    ncols = len(v) + 1
    M = [[0] * ncols for i in range(0, nrows)]
    for i in range(0, nrows):
        M[i][0] = i
    for j in range(0, ncols):
        M[0][j] = j
    for i in range(1, nrows):
        for j in range(1, ncols):
            candidates = [M[i-1][j] + 1, M[i][j-1] + 1]
            if (u[i-1] == v[j-1]):
                candidates.append(M[i-1][j-1])
            else:
                candidates.append(M[i-1][j-1] + 1)
            M[i][j] = min(candidates)
    return M[-1][-1]

def find_nearest(word, dictionary, keep_first=False):
    if keep_first:
        dictionary = [w for w in dictionary if w[0] == word[0]]
    distances = map(lambda x: edit_distance(word, x), dictionary)
    # [(45, "aaa"), ...] 
    return min(zip(distances, dictionary))[1]

In [9]:
corpus = read_movie_descriptions()
ir = IRsystem.from_corpus(corpus)

ID: 0
ID: 1000
ID: 2000
ID: 3000
ID: 4000
ID: 5000
ID: 6000
ID: 7000
ID: 8000
ID: 9000
ID: 10000
ID: 11000
ID: 12000
ID: 13000
ID: 14000
ID: 15000
ID: 16000
ID: 17000
ID: 18000
ID: 19000
ID: 20000
ID: 21000
ID: 22000
ID: 23000
ID: 24000
ID: 25000
ID: 26000
ID: 27000
ID: 28000
ID: 29000
ID: 30000
ID: 31000
ID: 32000
ID: 33000
ID: 34000
ID: 35000
ID: 36000
ID: 37000
ID: 38000
ID: 39000
ID: 40000
ID: 41000
ID: 42000


In [10]:
query_weighted(ir, "king")

(Esther, 121.13149441461108)
(The King and I, 104.3076757459151)
(Ready to Rumble, 97.57814827843671)
(King, 84.11909334347992)
(A Frozen Flower, 77.38956587600153)
(No Time for Sergeants, 77.38956587600153)
(Shishkabugs, 70.66003840852314)
(Esther... The Girl Who Became Queen, 70.66003840852314)
(The King and the Clown, 63.930510941044744)
(One Night with the King, 63.930510941044744)
(Victor/Victoria, 60.56574720730554)
(Agent X44, 60.56574720730554)
(Barbie in the Nutcracker, 57.200983473566346)
(Super Hornio Brothers, 57.200983473566346)
(Merlin, 57.200983473566346)
(Anna and the King, 57.200983473566346)
(Henry VIII, 53.83621973982715)
(Naresuan, 53.83621973982715)
(The Nutcracker Prince, 53.83621973982715)
(Scooby-Doo and the Goblin King, 50.471456006087955)
(Khan Kluay 2, 50.471456006087955)
(The Court Jester, 50.471456006087955)
(King Klunk, 50.471456006087955)
(The Prince and the Pauper, 50.471456006087955)
(The Legend of Zu, 50.471456006087955)
(Hitman, 50.471456006087955)
(T

(The Flintstones in Viva Rock Vegas, 3.364763733739197)
(The Proud General, 3.364763733739197)
(Life of Brian, 3.364763733739197)
(The Other Conquest, 3.364763733739197)
(Hercules in the Haunted World, 3.364763733739197)
(Norwegian Ninja, 3.364763733739197)
(Seetha Kalyanam, 3.364763733739197)
(Kung Fu Mahjong 2, 3.364763733739197)
(Sharpe's Honour, 3.364763733739197)
(Return to Halloweentown, 3.364763733739197)
(The Librarian: Quest for the Spear, 3.364763733739197)
(Heart and Souls, 3.364763733739197)
(Django Unchained, 3.364763733739197)
(Theatre of Blood, 3.364763733739197)
(Doraemon: Nobita and Fantastic Three Musketeers, 3.364763733739197)
(Meet Me at the Fair, 3.364763733739197)
(Wog Boy 2: Kings of Mykonos, 3.364763733739197)
(The Three Musketeers, 3.364763733739197)
(Imperium: Augustus, 3.364763733739197)
(Devta, 3.364763733739197)
(Mr. St. Nick, 3.364763733739197)
(ChromeSkull: Laid to Rest 2, 3.364763733739197)
(Godzilla vs. Biollante, 3.364763733739197)
(The Miracle, 3.3647